Load librarys

In [3]:
import numpy as np # linear algebra
import random as rnd
import math
from scipy.spatial import distance
from statistics import mode

Functions for loading the Data

In [4]:
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True

Loading the data

In [5]:
# Load the data
# data = loadCSV('./kmeans_data/data.csv')
data = loadCSV('./kmeans_data/data.csv')

Code For K-Means using Euclidean Distance

In [6]:
def assignPointsEuclidean(data,centroids):
    labels = []
    for point in data:
        label = 0
        minDistance = float('inf')
        for i , center in enumerate(centroids):
            dist = math.dist(point,center)
            if dist < minDistance:
                label = i
                minDistance = dist
        
        labels.append(label)
    
    return labels

def computeNewCentroids(data, labels, centroids):
    numCenters = len(centroids)
    orderedPoints = [[] for _ in range(numCenters)]
    newCentroids = []
    for i , point in enumerate(data):
        label = labels[i]
        orderedPoints[label].append(point)

    for list in orderedPoints:
        x = np.array(list)
        newCenter = np.average(x, axis=0)
        newCentroids.append(newCenter.tolist())
    
    return newCentroids

def kMeansEuclidean(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    while True:
        labels = assignPointsEuclidean(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        if centroids == newCentroids:
            break
        centroids = newCentroids
    
    return centroids, labels


Code for Kmeans using Cosine Similarity Distance

In [7]:
def assignPointsCosine(data,centroids):
    labels = []
    for point in data:
        label = 0
        minDistance = float('inf')
        for i , center in enumerate(centroids):
            
            dist = distance.cosine(point,center)
            if dist < minDistance:
                label = i
                minDistance = dist
        
        labels.append(label)
    
    return labels


def kMeansCosine(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    while True:
        labels = assignPointsCosine(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        if centroids == newCentroids:
            break
        centroids = newCentroids
    
    return centroids, labels


Code for K-means using Generalized Jaccard Dist

In [8]:
def generalizedJaccardDist(A, B):
    min = []
    max = []
    for i in range(len(A)):
        x = A[i]
        y = B[i]
        if x > y:
            min.append(y)
            max.append(x)
        else:
            min.append(x)
            max.append(y)
    
    return 1 - (sum(min) / sum(max))

def assignPointsJaccard(data,centroids):
    labels = []
    for point in data:
        label = 0
        minDistance = float('inf')
        for i , center in enumerate(centroids):
            
            dist = generalizedJaccardDist(point,center)
            if dist < minDistance:
                label = i
                minDistance = dist
        
        labels.append(label)
    
    return labels


def kMeansJaccard(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    while True:
        labels = assignPointsJaccard(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        if centroids == newCentroids:
            break
        centroids = newCentroids
    
    return centroids, labels

In [9]:
euclideanCentroids, euclideanLabels = kMeansEuclidean(data,10)

In [10]:
cosineCentroids, cosineLabels = kMeansCosine(data,10)

In [11]:
jaccardCentroids, jaccardLabels = kMeansJaccard(data, 10)

Creating SSE functions for each of the types of distances

In [12]:
def sSE(data,labels,centroids):
    sse = 0.0
    for i, point in enumerate(data):
        center = centroids[labels[i]]
        sse += math.dist(point,center) * math.dist(point,center)
    return sse

Comparing SSEs

In [13]:
print('SSE of k-means euclidean:', sSE(data,euclideanLabels,euclideanCentroids))
print('SSE of k-means cosine:', sSE(data,cosineLabels,cosineCentroids))
print('SSE of k-means jaccard:', sSE(data,jaccardLabels,jaccardCentroids))

SSE of k-means euclidean: 25316245969.759285
SSE of k-means cosine: 25678352105.430767
SSE of k-means jaccard: 25601961159.162224


Import Labels csv

In [14]:
f = open('./kmeans_data/label.csv')
trueLabels = []
lines = f.readlines()
for line in lines:
    trueLabels.append(int(line))

In [15]:

def getTrueLabelsOfClusters(data, tmplabels, trueLabels):
    tmp = [[] for _ in range(10)]
    l = []
    labels = []
    for i, lbl in enumerate(tmplabels):
        tmp[lbl].append(trueLabels[i])
    
    for x in tmp:
        l.append(mode(x))
    
    for x in tmplabels:
        labels.append(l[x])

    return labels


euclideanLabels = getTrueLabelsOfClusters(data,euclideanLabels, trueLabels)
cosineLabels = getTrueLabelsOfClusters(data,cosineLabels, trueLabels)
jaccardLabels = getTrueLabelsOfClusters(data,jaccardLabels, trueLabels)

In [16]:
def predictAccuracy(labels, trueLabels):
    N = len(trueLabels)
    numCorrect = 0
    for L, TL in zip(labels,trueLabels):
        if L == TL:
            numCorrect += 1
    
    return numCorrect/N

print('Accuracy of Euclidean:',predictAccuracy(euclideanLabels,trueLabels))
print('Accuracy of Cosine:',predictAccuracy(cosineLabels,trueLabels))
print('Accuracy of Jaccard:',predictAccuracy(jaccardLabels,trueLabels))

Accuracy of Euclidean: 0.1654
Accuracy of Cosine: 0.155
Accuracy of Jaccard: 0.1642


In [17]:
def kMeansEuclideanQ3(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsEuclidean(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if centroids == newCentroids:
            print('Break from centroids not changing')
            break
        if prevSSE <= newSSE:
            print('prevSSE',prevSSE)
            print('new SSE', newSSE)
            print('Break from SSE not improving')
            break
        if iterations == 500:
            print('Break from reaching max iterations')
            break
        
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1

    print('Num of iterations', iterations)
    
    return centroids, labels

def kMeansCosineQ3(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsCosine(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if centroids == newCentroids:
            print('Break from centroids not changing')
            break
        if prevSSE <= newSSE:
            print('Break from SSE not improving')
            break
        if iterations == 500:
            print('Break from reaching max iterations')
            break
        
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1

    print('Num of iterations', iterations)

    return centroids, labels

def kMeansJaccardQ3(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsJaccard(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if centroids == newCentroids:
            print('Break from centroids not changing')
            break
        if prevSSE <= newSSE:
            print('Break from SSE not improving')
            break
        if iterations == 500:
            print('Break from reaching max iterations')
            break
            
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1
    
    print('Num of iterations', iterations)

    return centroids, labels

In [18]:
euclideanCentroidsq3, euclideanLabelsq3 = kMeansEuclideanQ3(data,10)

Break from centroids not changing
Num of iterations 34


In [19]:
cosineCentroidsq3, cosineLabelsq3 = kMeansCosineQ3(data,10)

Break from SSE not improving
Num of iterations 34


In [20]:
jaccardCentroidsq3, jaccardLabelsq3 = kMeansJaccardQ3(data, 10)

Break from SSE not improving
Num of iterations 32


Code for Q4

In [24]:

def kMeansEuclideanQ4b(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsEuclidean(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if prevSSE <= newSSE:
            print('Break from SSE not improving')
            break

        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1

    print('Num of iterations', iterations)
    print('Euclidean SSE:', newSSE)
    
    return centroids, labels

def kMeansEuclideanQ4c(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsEuclidean(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if iterations == 100:
            print('Break from reaching max iterations')
            break
        
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1
    print('Num of iterations', iterations)
    print('Euclidean SSE:', newSSE)
    return centroids, labels


def kMeansCosineQ4b(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsCosine(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if prevSSE <= newSSE:
            print('Break from SSE not improving')
            break
        
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1

    print('Num of iterations', iterations)
    print('Cosine SSE:', newSSE)

    return centroids, labels

def kMeansCosineQ4c(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsCosine(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if iterations == 100:
            print('Break from reaching max iterations')
            break
        
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1

    print('Num of iterations', iterations)
    print('Cosine SSE:', newSSE)

    return centroids, labels

def kMeansJaccardQ4b(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsJaccard(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)
        if prevSSE <= newSSE:
            print('Break from SSE not improving')
            break
            
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1
    
    print('Num of iterations', iterations)
    print('Jaccard SSE:', newSSE)

    return centroids, labels

def kMeansJaccardQ4c(data,numClusters):
    centroids = rnd.sample(data, numClusters)
    prevSSE = float('inf')
    iterations = 0
    while True:
        labels = assignPointsJaccard(data, centroids)
        newCentroids = computeNewCentroids(data,labels,centroids)
        newSSE = sSE(data,labels,newCentroids)

        if iterations == 100:
            break
            
        centroids = newCentroids
        prevSSE = newSSE
        iterations += 1
    
    print('Num of iterations', iterations)
    print('Jaccard SSE:', newSSE)

    return centroids, labels

In [22]:
x, y = kMeansEuclideanQ4b(data,10)
x, y = kMeansCosineQ4b(data,10)
x, y = kMeansJaccardQ4b(data,10)

Break from SSE not improving
Num of iterations 51
Euclidean SSE: 25367259439.050186
Break from SSE not improving
Num of iterations 23
Cosine SSE: 25496934056.703373
Break from SSE not improving
Num of iterations 17
Jaccard SSE: 25457448407.770412


In [25]:
x, y = kMeansEuclideanQ4c(data,10)
x, y = kMeansCosineQ4c(data,10)
x, y = kMeansJaccardQ4c(data,10)

Break from reaching max iterations
Num of iterations 100
Euclidean SSE: 25404933340.755253
Break from reaching max iterations
Num of iterations 100
Cosine SSE: 25422699865.141636
Num of iterations 100
Jaccard SSE: 25452099483.324234
